# Nimbus ideal vs. payoff ideal

There is some issues that cause the results attained by Nimbus method to be higher than the ideal values attained by calculating the payoff table for the clusters.

In [1]:
from interactiveBoreal import *
import gradutil
from BorealWeights import BorealWeightedProblem

In [2]:
kehys = ReferenceFrame()
nclust = 150
kehys.cluster(nclust=nclust, seedn=2, outdata=kehys.x_stack)
data = kehys.centers
weights_norm = kehys.weights/len(kehys.x_norm)
ideal = kehys.ideal
nadir = kehys.nadir

In [3]:
solver_name = 'cplex'

Now we try to calculate ideal and possible nadir values for all the objectives using nimbus scalarization:

In [4]:
nimbus_ref = np.array([[kehys.ideal[0], kehys.nadir[1], kehys.nadir[2], kehys.nadir[3]],
                       [kehys.nadir[0], kehys.ideal[1], kehys.nadir[2], kehys.nadir[3]],
                       [kehys.nadir[0], kehys.nadir[1], kehys.ideal[2], kehys.nadir[3]],
                       [kehys.nadir[0], kehys.nadir[1], kehys.nadir[2], kehys.ideal[3]]])
stay = np.array([], dtype=int)
detoriate = np.array([], dtype=int)
nimbus_res = []
nimbus_problems = []
for i in range(len(nimbus_ref)):
    minmax = np.array([i], dtype=int)
    nimbus_problems.append(NIMBUS(kehys.ideal, kehys.nadir, nimbus_ref[i], kehys.centers, minmax, stay, detoriate, np.array([0,0,0,0]), weights=kehys.weights))
    nimbus_solver = Solver(nimbus_problems[i].model, solver=solver_name)
    res = nimbus_solver.solve() 
    nimbus_res.append([gradutil.model_to_real_values(kehys.x_stack[:,:,j], nimbus_problems[i].model, kehys.xtoc) for j in range(len(nimbus_ref))])
nimbus_res = np.array(nimbus_res)

In [5]:
for k in range(len(nimbus_res)):
    print(nimbus_res[k])

[  2.37791598e+08   2.92580481e+06   1.02020408e+05   1.29577928e+04]
[  1.56270534e+08   3.52674505e+06   1.70134894e+05   1.61041905e+04]
[  1.73775294e+08   3.44637343e+06   1.60506959e+05   1.55639002e+04]
[  1.72927424e+08   3.43287166e+06   1.55198829e+05   1.51836908e+04]


As we can see, the greatest values column wise are not at the diagonal. This indicates there is something weird going on...

In [6]:
for i in range(len(nimbus_problems)):
    print(gradutil.cluster_to_value(kehys.x_stack, gradutil.res_to_list(nimbus_problems[i].model), kehys.weights))

[  3.13866023e+08   3.01196947e+06   9.97130032e+04   1.58995465e+04]
[  2.26515486e+08   3.58976924e+06   1.44846447e+05   1.78207300e+04]
[  2.60536118e+08   3.37229210e+06   1.50975670e+05   1.88416660e+04]
[  2.45705661e+08   3.46626409e+06   1.47728286e+05   1.88031501e+04]


Now when we print the values the nimbus is internally using, we see that the biggest values actually are at the diagonal. This means the clustering is biasing the results, because not all objectives are responding the same way to the clustering!

However the ideal of the attained results is:


In [7]:
nimbus_ideal = np.max(nimbus_res, axis=0)
nimbus_ideal

array([  2.37791598e+08,   3.52674505e+06,   1.70134894e+05,
         1.61041905e+04])

And the nadir

In [8]:
nimbus_nadir = np.min(nimbus_res, axis=0)
nimbus_nadir

array([  1.56270534e+08,   2.92580481e+06,   1.02020408e+05,
         1.29577928e+04])

However if we are using payoff-table for the same data, the results are different:

In [9]:
data = np.array([kehys.x_stack[np.argmin(kehys.dist[kehys.xtoc == i]), :, :] for i in range(nclust)])
solver = SolverFactory(solver_name)
problems = []
for i in range(np.shape(data)[-1]):
    problems.append(BorealWeightedProblem(data[:, :, i], kehys.weights))

for j in range(len(problems)):
    solver.solve(problems[j].model)

payoff = [[gradutil.model_to_real_values(kehys.x_stack[:,:,i], problems[j].model, kehys.xtoc)
           for i in range(np.shape(data)[-1])] 
          for j in range(len(problems))]

In [18]:
gradutil.calc_ideal_n_nadir(kehys.centers, kehys.xtoc, kehys.weights)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 150 but corresponding boolean dimension is 29666

In [10]:
for k in range(len(payoff)):
    print(payoff[k])

[239721168.69964999, 2882099.780669, 89177.025272300045, 12428.763925952006]
[81413759.163200006, 3990254.857198, 186947.93321125791, 17191.647986277214]
[104287441.70060003, 3862780.9707499999, 174174.78411868005, 16271.656338333207]
[101876583.31060001, 3868167.1460200017, 170279.00133698006, 16131.761410395002]


Lets see if here is something similar going on than with the nimbus ideals

In [11]:
for i in range(len(nimbus_problems)):
    print(gradutil.cluster_to_value(kehys.x_stack, gradutil.res_to_list(problems[i].model), weights_norm))

[  1.07859746e+04   1.00283163e+02   3.82328958e+00   5.52468680e-01]
[  4.75830003e+03   1.37730525e+02   5.28686423e+00   6.10833469e-01]
[  4.68713528e+03   1.40296738e+02   6.02469544e+00   7.21618201e-01]
[  4.69414726e+03   1.39508802e+02   5.97691701e+00   7.33569853e-01]


It almost looks like that we are minimizing the last objectives...

In [12]:
payoff_ideal = np.max(payoff, axis=0)
payoff_ideal

array([  2.39721169e+08,   3.99025486e+06,   1.86947933e+05,
         1.71916480e+04])

In [13]:
payoff_nadir = np.min(payoff, axis=0)
payoff_nadir

array([  8.14137592e+07,   2.88209978e+06,   8.91770253e+04,
         1.24287639e+04])

Compare the differences also:


In [14]:
payoff_ideal - nimbus_ideal

array([  1.92957050e+06,   4.63509805e+05,   1.68130394e+04,
         1.08745747e+03])

In [15]:
payoff_nadir - nimbus_nadir

array([ -7.48567752e+07,  -4.37050328e+04,  -1.28433830e+04,
        -5.29028855e+02])

Absolutely the values are quite big, but the directions of the differences are good.

Relative differences:

In [16]:
(payoff_ideal - nimbus_ideal)/payoff_ideal

array([ 0.00804923,  0.11616045,  0.08993434,  0.06325499])

In [17]:
(payoff_nadir - nimbus_nadir)/payoff_nadir

array([-0.91946098, -0.0151643 , -0.14402121, -0.04256488])

The differences are much bigger for the nadir, but it is no surprise because of the payoff table -method

The ideals are much closer so probably there is no need to worry. There used to be greater differences, but setting the closest point as the centroid helped a lot and also the payoff table had to be fixed a bit.